In It to Win IT

Token + API

In [3]:
import os

# Set the environment variable manually or use a .env file with dotenv
os.environ['TOKEN'] = 'rovjhwxiopitrddsxhrdvakheleadupk' 

# Retrieve the API token from environment variables
api_token = os.getenv('TOKEN')
api_url = 'https://codeweekend.dev:3721/api/'
files_url = 'https://codeweekend.dev:81/'

print('API TOKEN:', api_token)
print('API URL:', api_url)


API TOKEN: rovjhwxiopitrddsxhrdvakheleadupk
API URL: https://codeweekend.dev:3721/api/


API helping functions

In [5]:
import requests
import json
import time
import os

headers = {
    'authorization': f'Bearer {api_token}'
}

def show(inner_json):
    print(json.dumps(inner_json, indent=2))

def get_scoreboard():
    return requests.get(api_url + 'scoreboard', headers=headers).json()

def get_team_dashboard():
    return requests.get(api_url + 'team_dashboard', headers=headers).json()

def get_test(task_id):
    task_id_padded = '{:03d}'.format(task_id)
    url = f'{files_url}{task_id_padded}.json'
    return requests.get(url, headers=headers).content

# Returns at most 50 submissions
def get_team_submissions(offset=0, task_id=None):
    url = f'{api_url}team_submissions?offset={offset}'
    if task_id is not None:
      url += f'&task_id={task_id}'
    return requests.get(url, headers=headers).json()

def get_submission_info(submission_id, wait=False):
    url = f'{api_url}submission_info/{submission_id}'
    res = requests.get(url, headers=headers).json()
    if 'Pending' in res and wait:
        print('Submission is in Pending state, waiting...')
        time.sleep(1)
        return get_submission_info(submission_id)
    return res

# Returns submission_id
def submit(task_id, solution):
    res = requests.post(url = f'{api_url}submit/{task_id}',
                        headers=headers, files={'file': solution})
    if res.status_code == 200:
        return res.text
    print(f'Error: {res.text}')
    return None

def download_submission(submission_id):
    import urllib.request
    url = f'{api_url}download_submission/{submission_id}'
    opener = urllib.request.build_opener()
    opener.addheaders = headers.items()
    urllib.request.install_opener(opener)
    try:
        file, _ = urllib.request.urlretrieve(url, "downloaded.txt")
    except Exception as e:
        print('Failed to download submission: ', e)
        return None
    content = open(file, "r").read()
    os.remove(file)
    return content

def update_display_name(new_name):
  url = api_url + 'update_user'
  data = {
      'display_name': new_name,
      'email': "",
      'team_members': ""
  }
  return requests.post(url, json=data, headers=headers).content

# show(get_scoreboard())
# show(get_submission_info(427))
# show(get_team_dashboard())
# show(get_team_submissions())
# download_submission(476)
# get_test(1)
# update_display_name('Test 123')

Read input

In [16]:
import json

def read_heropath_input(file_path):
    with open(file_path, 'r') as file:
        data = json.load(file)
    return data

# Example usage:
file_path = 'heropath_inputs_day1/001.json'
input_data = read_heropath_input(file_path)

Game logic

In [22]:
import json
import math

def distance(x1, y1, x2, y2):
    return math.sqrt((x1 - x2) ** 2 + (y1 - y2) ** 2)

def can_move_to(x1, y1, x2, y2, speed):
    return distance(x1, y1, x2, y2) <= speed

def can_attack(hero_x, hero_y, monster_x, monster_y, attack_range):
    return distance(hero_x, hero_y, monster_x, monster_y) <= attack_range

def find_nearest_monster(hero_x, hero_y, monsters):
    nearest_monster = None
    min_distance = float('inf')
    for i, monster in enumerate(monsters):
        if monster['hp'] > 0:
            dist = distance(hero_x, hero_y, monster['x'], monster['y'])
            if dist < min_distance:
                min_distance = dist
                nearest_monster = i
    return nearest_monster

def maximize_gold(input_data):
    hero = input_data['hero']
    hero_x = input_data['start_x']
    hero_y = input_data['start_y']
    monsters = input_data['monsters']
    num_turns = input_data['num_turns']
    
    moves = []
    
    for turn in range(num_turns):
        nearest_monster_idx = find_nearest_monster(hero_x, hero_y, monsters)
        if nearest_monster_idx is None:
            break

        monster = monsters[nearest_monster_idx]
        if can_attack(hero_x, hero_y, monster['x'], monster['y'], hero['base_range']):
            moves.append({
                "type": "attack",
                "target_id": nearest_monster_idx
            })
            monsters[nearest_monster_idx]['hp'] = 0  # Assume attack kills the monster for simplicity
        else:
            if can_move_to(hero_x, hero_y, monster['x'], monster['y'], hero['base_speed']):
                hero_x, hero_y = monster['x'], monster['y']
            else:
                direction_x = (monster['x'] - hero_x) / distance(hero_x, hero_y, monster['x'], monster['y'])
                direction_y = (monster['y'] - hero_y) / distance(hero_x, hero_y, monster['x'], monster['y'])
                hero_x += direction_x * hero['base_speed']
                hero_y += direction_y * hero['base_speed']
                hero_x = int(round(hero_x))
                hero_y = int(round(hero_y))
            moves.append({
                "type": "move",
                "target_x": hero_x,
                "target_y": hero_y
            })

    return {
        "moves": moves
    }

print(input_data)
output = maximize_gold(input_data)
print(json.dumps(output, indent=4))

# Save to output.json
with open('output.json', 'w') as f:
    json.dump(output, f, indent=4)


{'hero': {'base_speed': 10, 'base_power': 50, 'base_range': 10, 'level_speed_coeff': 50, 'level_power_coeff': 50, 'level_range_coeff': 50}, 'start_x': 62, 'start_y': 53, 'width': 100, 'height': 100, 'num_turns': 12, 'monsters': [{'x': 30, 'y': 24, 'hp': 0, 'gold': 206, 'exp': 64}, {'x': 81, 'y': 56, 'hp': 0, 'gold': 170, 'exp': 44}, {'x': 32, 'y': 50, 'hp': 0, 'gold': 170, 'exp': 47}, {'x': 63, 'y': 46, 'hp': 0, 'gold': 141, 'exp': 29}, {'x': 50, 'y': 24, 'hp': 0, 'gold': 183, 'exp': 80}, {'x': 40, 'y': 14, 'hp': 0, 'gold': 157, 'exp': 59}, {'x': 40, 'y': 34, 'hp': 0, 'gold': 154, 'exp': 43}]}
{
    "moves": []
}


Block to create solution and submit

In [24]:
def example():
    with open('output.json', 'r') as file:
        solution = file.read()
    submission_id = submit(1, solution)
    print(f'Submission_id: {submission_id}')
    info = get_submission_info(submission_id, wait=True)
    print(f'Result: {info}')

example()

Submission_id: 3736
Result: {'Ok': 0.0}
